In [231]:
import json

In [232]:
# Load the correct answer Harrison provided me with previously (first fragment).
gold_standard = json.load(open("./11gs/grid_0.right_answer.json"))

In [233]:
# Load the json produced by running my make_grid.py script.
jdd_output = json.load(open("./jdd_version_grid.json"))

In [234]:
# They are the same size (1 rotation, 4 parent + 5 receptor channels= 9, 24 x 24 x 24 grid)
len(gold_standard), len(jdd_output), 1 * 9 * 24 * 24 * 24

(124416, 124416, 124416)

In [235]:
# Quick check of the value ranges.
min(gold_standard), max(gold_standard), min(jdd_output), max(jdd_output)

(0.0, 1.7793281078338623, 0, 8.896640596387922)

In [236]:
# The max value of the jdd_output is exactly 5 times bigger 
# than the max value of the gold standard. Can't for the life 
# of me figure out why...
print(max(jdd_output) / max(gold_standard))

5.000000032157425


In [237]:
# Let's numpiphy the arrays and reshape them to ease comparison.
import numpy as np
gold_standard = np.array(gold_standard).reshape(1, 9, 24, 24, 24)
jdd_output = np.array(jdd_output).reshape(1, 9, 24, 24, 24)

In [238]:
# Any chance they are identical except scaled by 5? Nope...
diff = 5 * gold_standard[0] - jdd_output[0]
np.min(diff), np.max(diff)

(-6.850364752509347, 4.892328977584839)

In [239]:
# Let's look at them channel by channel to see if some are equivalent. First, let's identify the 
# channels that are totally empty, to ignore those.
channel_names = {
    0: "parent carbon",
    1: "parent nitrogen",
    2: "parent oxygen",
    3: "parent other (not including hydrogen)",
    4: "receptor carbon",
    5: "receptor nitrogen",
    6: "receptor oxygen",
    7: "receptor sulfur",
    8: "receptor other (not including hydrogen)",
}

channels_to_consider = list(range(9))
tmp = channels_to_consider[:]
for i in channels_to_consider:
    if np.min(gold_standard[0][i]) == np.max(gold_standard[0][i]) == np.min(jdd_output[0][i]) == np.max(jdd_output[0][i]) == 0.0:
        print("Ignoring " + channel_names[i])
        tmp.remove(i)
channels_to_consider = tmp

Ignoring receptor other (not including hydrogen)


In [240]:
# Now do the comparison of the remaining channels
tmp = channels_to_consider[:]
for i in channels_to_consider:
    diff = 5 * gold_standard[0][i] - jdd_output[0][i]
    if np.fabs(np.min(diff)) < 1e-6 and np.fabs(np.max(diff)) < 1e-6:
        print("PASS: Channel \"" + channel_names[i] + "\" is the same if you scale Harrison's by 5.")
        tmp.remove(i)
    else:
        print("FAIL: Channel \"" + channel_names[i] + "\" is NOT the same if you scale Harrison's by 5.")
channels_to_consider = tmp

FAIL: Channel "parent carbon" is NOT the same if you scale Harrison's by 5.
FAIL: Channel "parent nitrogen" is NOT the same if you scale Harrison's by 5.
FAIL: Channel "parent oxygen" is NOT the same if you scale Harrison's by 5.
FAIL: Channel "parent other (not including hydrogen)" is NOT the same if you scale Harrison's by 5.
PASS: Channel "receptor carbon" is the same if you scale Harrison's by 5.
PASS: Channel "receptor nitrogen" is the same if you scale Harrison's by 5.
PASS: Channel "receptor oxygen" is the same if you scale Harrison's by 5.
PASS: Channel "receptor sulfur" is the same if you scale Harrison's by 5.


In [243]:
# So it's the parent ones that differ. Turns out that for two atoms, Harrison gold standard is all 0.0

tmp = channels_to_consider[:]
for i in channels_to_consider:
    mn1 = np.min(gold_standard[0][i])
    mx1 = np.max(gold_standard[0][i])
    mn2 = np.min(jdd_output[0][i])
    mx2 = np.max(jdd_output[0][i])
    print(channel_names[i], ": GOLD", mn1, "-", mx1, "; JDD: ", mn2, "-", mx2)
                
# But I think there are nitrogen and oxygen atoms in the parent molecule, so confused...

parent carbon : GOLD 0.0 - 0.9738303422927856 ; JDD:  0.0 - 6.850364752509347
parent nitrogen : GOLD 0.0 - 0.0 ; JDD:  0.0 - 3.5999865747605555
parent oxygen : GOLD 0.0 - 0.0 ; JDD:  0.0 - 3.9138633001190715
parent other (not including hydrogen) : GOLD 0.0 - 0.9784657955169678 ; JDD:  0.0 - 3.89532126395043


In [244]:
# Can the indexes of the max values tell us anything?
def find_max_vals(channel):
    mx1, mx2 = np.argmax(gold_standard[0][channel]), np.argmax(jdd_output[0][channel])
    idx1 = np.unravel_index(mx1, (24, 24, 24))
    idx2 = np.unravel_index(mx2, (24, 24, 24))
    print(channel_names[channel], "GOLD:", idx1, "JDD:", idx2)

for i in channels_to_consider:
    find_max_vals(i)

parent carbon GOLD: (3, 0, 18) JDD: (2, 4, 14)
parent nitrogen GOLD: (0, 0, 0) JDD: (3, 8, 18)
parent oxygen GOLD: (0, 0, 0) JDD: (2, 12, 19)
parent other (not including hydrogen) GOLD: (2, 12, 19) JDD: (3, 0, 18)


In [245]:
# What a coincidence... the max value of parent carbon GOLD matches parent other 
# JDD, and parent other GOLD matches oxygen JDD.

# Apparently "4" is a magical number like "5" was for the receptor. Normalized by 
# number of channels, maybe?
print(jdd_output[0][3][3][0][18] / gold_standard[0][0][3][0][18])
print(jdd_output[0][2][2][12][19] / gold_standard[0][3][2][12][19])


3.9999998919517004
4.000000120649287


In [247]:
# But the 4 factor doesn't hold across the board, because my version 
# occasionally has some entries that Harrison's doesn't.

jdd_idx = 3
har_idx = 0

for jdd, har in zip(jdd_output[0][jdd_idx].flatten(), gold_standard[0][har_idx].flatten()):
    if jdd != 0 or har != 0:  # filter out ones where both are 0
        print(jdd, har, har/jdd)

0.6056528373325465 0.1514132022857666 0.24999998836401055
0.8857091104119671 0.22142727673053741 0.24999999901496514
0.6212765630861987 0.15531913936138153 0.24999999773020867
0.7469061932627247 0.18672655522823334 0.25000000925491345
0.601513995836313 0.1503784954547882 0.24999999417421692
1.8339461926140126 0.45848655700683594 0.25000000482747686
2.6819701827165576 0.6704925298690796 0.24999999410505758
1.8812555926434942 0.4703139066696167 0.2500000045229065
0.6329481953321799 0.1582370549440384 0.2500000096548082
1.5465413568308852 0.38663533329963684 0.24999999617980828
2.2616682114574 0.5654170513153076 0.2499999993150886
1.5864367169058515 0.39660918712615967 0.2500000049795222
0.6255513850150913 0.15638785064220428 0.2500000070153013
0.9148088254302585 0.22870220243930817 0.2499999957168576
0.6416884237307653 0.16042210161685944 0.2499999932742563
0.873645155954595 0.21841128170490265 0.24999999166280948
2.6636424398254706 0.6659106016159058 0.24999999686877572
3.89532126395043

In [248]:
# Same story with the other pairing, but this time Harrison's has some values that mine doesn't.

jdd_idx = 2
har_idx = 3

for jdd, har in zip(jdd_output[0][jdd_idx].flatten(), gold_standard[0][har_idx].flatten()):
    if jdd != 0 or har != 0:  # filter out ones where both are 0
        print(jdd, har, har/jdd)

0.6942345025554865 0.17355862259864807 0.24999999562075417
1.2122065325054496 0.303051620721817 0.24999998976697035
1.015251288813981 0.2538128197193146 0.24999999755313712
0.543245754099971 0.1358114331960678 0.24999999019058153
1.612044910772472 0.4030112326145172 0.25000000305289216
2.8148001350226353 0.7037000060081482 0.24999999014228033
2.357460868428913 0.5893652439117432 0.25000001137007843
0.9470391331461001 0.23675978183746338 0.2499999984699031
1.795455241137821 0.44886380434036255 0.24999999668936737
3.135053881817781 0.7837634682655334 0.2499999993017945
2.625680862681267 0.6564202308654785 0.250000005787132
1.0547884638989302 0.2636971175670624 0.25000000150962004
0.9591765054285404 0.23979412019252777 0.24999999357302094
1.6748231633925392 0.4187057912349701 0.2500000002309708
1.4027035241720576 0.35067588090896606 0.24999999990443575
0.5634940318132123 0.1408735066652298 0.24999999771413145
0.7996948453045586 0.19992370903491974 0.24999999713488225
1.7355633800308639 0.

/Users/jdurrant/anaconda/envs/Python36/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in double_scalars
  
